In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scanpy as sc
import anndata

from scipy.stats import pearsonr,hypergeom,spearmanr

# Loading in the GT results

In [2]:
# Load in the set of 316 drugs used in the screens
cs_drugs = pd.read_csv("../screening_methods/mahalanobis_calcs_on_compressed_15Nov2021/mahala_calculations/CS_run1_2.0x5.0r_randommahala_table.csv").Metadata_perturbation.values
gt_mahala = pd.read_csv("../../../BEM/Manu_analysis/2_OG316_analysis/GT316_mahala_empirical_table.csv")

# Need to update the name of one drug with a weird name that got annotated in two ways
gt_mahala.Metadata_perturbation.values[gt_mahala.Metadata_perturbation.values=='Methyldopa Sesquihydrate (L-A-Methyl-Dopa Sesquihyrate)']='Methyldopa Sesquihydrate (L-_-Methyl-Dopa Sesquihyrate)'

# Filter down to OG 316 and remove the DMSO wells
gt_mahala = gt_mahala.loc[gt_mahala.Metadata_perturbation.isin(cs_drugs)]
gt_mahala = gt_mahala.loc[gt_mahala.Metadata_perturbation!="DMSO"]


# rename columns where necesseary
gt_mahala.columns = ['Metadata_perturbation','gt_mahalanobis','gt_signif_mahala']
gt_mahala.index = gt_mahala.Metadata_perturbation.values
gt_mahala.sort_index(inplace=True)

gt_mahala.gt_signif_mahala = 'False'
# gt_mahala.gt_signif_mahala.loc[gt_mahala.gt_mahalanobis > np.percentile(gt_metrics.gt_mahalanobis,95)] = 'True'
# gt_mahala.gt_signif_mahala.loc[gt_mahala.gt_mahalanobis > np.mean(gt_metrics.gt_mahalanobis) + 2*np.std(gt_metrics.gt_mahalanobis)] = 'True'
sig_mahala_drugs = pd.read_csv("GT_316_sig_mahala.csv",index_col=0)
gt_mahala.gt_signif_mahala.loc[np.isin(gt_mahala.Metadata_perturbation,sig_mahala_drugs.Metadata_perturbation)] = 'True'

# Loading in the GT phenotyping results
gt_leiden = pd.read_csv("GT316_drug_clusters.csv",index_col=0)
# Need to update the name of one drug with a weird name that got annotated in two ways
gt_leiden.Metadata_perturbation.values[gt_leiden.Metadata_perturbation.values=='Methyldopa Sesquihydrate (L-A-Methyl-Dopa Sesquihyrate)']='Methyldopa Sesquihydrate (L-_-Methyl-Dopa Sesquihyrate)'
gt_leiden['gt_sig_leiden'] = True
gt_leiden.columns.values[1] = 'gt_leiden'

# Combining mahalanobis and phenotyping results
gt_metrics = gt_mahala.merge(gt_leiden,how='left')
gt_metrics.index = gt_metrics.Metadata_perturbation

/var/folders/7v/c1jnbc_12kq312b56vlhgmv40000gn/T/ipykernel_5486/331540485.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gt_mahala.gt_signif_mahala.loc[np.isin(gt_mahala.Metadata_perturbation,sig_mahala_drugs.Metadata_perturbation)] = 'True'


# Loading in the CS regression deconvolution results and assesing effect sizes for each compound in each screen

In [5]:
np.arange(0.0,1.01,step=0.01).round(2)

array([0.  , 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ,
       0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21,
       0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32,
       0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43,
       0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54,
       0.55, 0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65,
       0.66, 0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76,
       0.77, 0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87,
       0.88, 0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98,
       0.99, 1.  ])

In [6]:
data_read_path = "../../../cell_painting_data_lock/4_CS_OBJECTS_median_aggregated/"
data_read_suffix = "_PCH_rd_OG316"
save_path = "regression_summary_stats/"
sc.settings.figdir = save_path

regression_read_path = "../regression_deconvolution_December2021/decon_out/raw_rd_OG316_raw_decon_out/"

permute_thresholds = np.arange(0.0,1.01,step=0.01).round(2).astype(str)
for permute_threshold in permute_thresholds:
    print(permute_threshold)
    runs = ['CS_run1','CS_run2','CS_run3']
    for run in runs:
        print(run)
        metadata = pd.read_csv(data_read_path+run+data_read_suffix+"_metadata.csv",index_col=0)
         # Layout all of the compression schemes
        compression_methods = metadata.groupby(['Metadata_compression','Metadata_replicates','Metadata_perturbation','Metadata_Plate']).size().reset_index().rename(columns={0:''})
        compression_methods = compression_methods.loc[np.isin(compression_methods['Metadata_perturbation'],['random','random1','random2'])]
        compression_methods.to_csv(run+"_compression_methods.csv",index=False)
        for i in range(compression_methods.shape[0]):
            print(i)
            compression = compression_methods['Metadata_compression'].iloc[i]
            replicates = compression_methods['Metadata_replicates'].iloc[i]
            scheme = compression_methods['Metadata_perturbation'].iloc[i]
            save_prefix = str(compression) + "x"+str(replicates) + "r" +"_"+str(scheme)



            # Loading in the permuted & non-permuted regression results
            model_coef = pd.read_csv(regression_read_path +run+"_"+str(compression)+"x_"+str(replicates)+"r"+"_"+scheme+"_model_coef.csv",index_col=0)
            model_coef.index.rename('Metadata_perturbation',inplace=True)
            model_coef = model_coef.loc[model_coef.index.values!="DMSO"]
            model_coef.sort_index(inplace=True)
            model_coef['regression_l1_norm'] = np.abs(model_coef).sum(1)

            permute_model_coef = pd.read_csv(regression_read_path +run+"_"+str(compression)+"x_"+str(replicates)+"r"+"_"+scheme+"_permute_"+permute_threshold+"_model_coef.csv",index_col=0)
            permute_model_coef.index.rename('Metadata_perturbation',inplace=True)
            permute_model_coef = permute_model_coef.loc[permute_model_coef.index.values!="DMSO"]
            permute_model_coef.sort_index(inplace=True)
            permute_model_coef['regression_l1_norm'] = np.abs(permute_model_coef).sum(1)


            # For now removing the reg leidens
            permute_model_coef['permute_reg_l1_norm'] = permute_model_coef.regression_l1_norm
            permute_model_coef['permute_reg_l1_scaled'] = permute_model_coef.regression_l1_norm /np.max(permute_model_coef.regression_l1_norm)
            reg_metrics = pd.concat([model_coef.regression_l1_norm,
                      permute_model_coef.permute_reg_l1_norm,permute_model_coef.permute_reg_l1_scaled],
                      sort=True,axis=1)
            reg_metrics.Metadata_perturbation = reg_metrics.index.values

            cs_metrics = pd.concat([reg_metrics,gt_metrics],sort=True,axis=1)
            cs_metrics.index = np.arange(316)
            cs_metrics['Metadata_run'] = run
            cs_metrics['Metadata_compresion'] = compression
            cs_metrics['Metadata_replicates'] = replicates
            cs_metrics['Metadata_scheme'] = scheme
            new_order = cs_metrics.columns.values[9:13].tolist()+cs_metrics.columns.values[0:4].tolist()+cs_metrics.columns.values[5:9].tolist()
            # cs_metrics = cs_metrics[new_order]
            cs_metrics = cs_metrics.loc[:,~cs_metrics.columns.duplicated()]

            if i ==0 and run =="CS_run1":
                combined_metrics = cs_metrics
            else:
                combined_metrics = pd.concat([combined_metrics,cs_metrics],axis=0)
    combined_metrics.to_csv(save_path+"regression_metrics_permute_"+permute_threshold+"_Dec102021.csv")     

0.0
CS_run1
0


/var/folders/7v/c1jnbc_12kq312b56vlhgmv40000gn/T/ipykernel_5486/245974346.py:48: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  reg_metrics.Metadata_perturbation = reg_metrics.index.values


1


/var/folders/7v/c1jnbc_12kq312b56vlhgmv40000gn/T/ipykernel_5486/245974346.py:48: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  reg_metrics.Metadata_perturbation = reg_metrics.index.values


2


...

/var/folders/7v/c1jnbc_12kq312b56vlhgmv40000gn/T/ipykernel_5486/245974346.py:48: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  reg_metrics.Metadata_perturbation = reg_metrics.index.values
/var/folders/7v/c1jnbc_12kq312b56vlhgmv40000gn/T/ipykernel_5486/245974346.py:48: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  reg_metrics.Metadata_perturbation = reg_metrics.index.values


5
6


/var/folders/7v/c1jnbc_12kq312b56vlhgmv40000gn/T/ipykernel_5486/245974346.py:48: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  reg_metrics.Metadata_perturbation = reg_metrics.index.values
/var/folders/7v/c1jnbc_12kq312b56vlhgmv40000gn/T/ipykernel_5486/245974346.py:48: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  reg_metrics.Metadata_perturbation = reg_metrics.index.values


7


/var/folders/7v/c1jnbc_12kq312b56vlhgmv40000gn/T/ipykernel_5486/245974346.py:48: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  reg_metrics.Metadata_perturbation = reg_metrics.index.values


# Calculate summary statistics over all compounds for each screen

In [35]:
data_read_path = "../../../cell_painting_data_lock/4_CS_OBJECTS_median_aggregated/"
data_read_suffix = "_PCH_rd_OG316"
save_path = "regression_summary_stats/"
permute_thresholds = np.arange(0.0,1.01,step=0.01).round(2).astype(str)
for permute_threshold in permute_thresholds:
    combined_metrics = pd.read_csv(save_path+"regression_metrics_permute_"+permute_threshold+"_Dec102021.csv",index_col=0)

    run_vals = []
    compression_vals = []
    rep_vals = []
    scheme_vals = []

    pearson_correlations = []
    spearman_correlations = []
    # top_5_mahala_percent = []
    # top_10_mahala_percent = []
    # top_32_mahala_percent = []
    # number_of_gt_leiden_in_cs = []
    # fraction_of_gt_leiden_in_cs = []
    # pvalue_gt_leiden_in_cs = []
    # percent_of_cs_that_is_gt_leiden_sig = []

    # cs_hit_nominate_num =[]
    # sample_savings = []

    # n_drugs_per_pool = []
    
    # num_top_hits = []
    # percent_top_sig_mahala = []
    # percent_top_sig_leiden = []
    # top_hit_thresholds = []

    tpr = []
    fpr = []
    tp = []
    fp = []
    tn = []
    fn = []
    precision = []
    recall = []


    for run in ['CS_run1','CS_run2','CS_run3']:
        print(run)
        metadata = pd.read_csv(data_read_path+run+data_read_suffix+"_metadata.csv",index_col=0)
         # Layout all of the compression schemes
        compression_methods = metadata.groupby(['Metadata_compression','Metadata_replicates','Metadata_perturbation','Metadata_Plate']).size().reset_index().rename(columns={0:''})
        compression_methods = compression_methods.loc[np.isin(compression_methods['Metadata_perturbation'],['random','random1','random2'])]
        compression_methods.to_csv(run+"_compression_methods.csv",index=False)

        for i in range(compression_methods.shape[0]):
        
            compression = compression_methods['Metadata_compression'].iloc[i]
            replicates = compression_methods['Metadata_replicates'].iloc[i]
            scheme = compression_methods['Metadata_perturbation'].iloc[i]
            print(str(compression)+"_"+str(replicates)+scheme)
            # Finding the drugs with at least one non zero coeficient
            cs_metrics = combined_metrics.loc[(combined_metrics.Metadata_run==run)&
                          (combined_metrics.Metadata_compresion==compression)&
                          (combined_metrics.Metadata_replicates==replicates)&
                                (combined_metrics.Metadata_scheme==scheme)&
                               (combined_metrics.permute_reg_l1_norm>0)]
            
            cs_metrics_all_perturbs = combined_metrics.loc[(combined_metrics.Metadata_run==run)&
              (combined_metrics.Metadata_compresion==compression)&
              (combined_metrics.Metadata_replicates==replicates)&
                    (combined_metrics.Metadata_scheme==scheme)]
            top_hit_threshold = np.mean(cs_metrics_all_perturbs.permute_reg_l1_norm) + 2*np.std(cs_metrics_all_perturbs.permute_reg_l1_norm)
            top_cs_all = cs_metrics_all_perturbs.loc[cs_metrics_all_perturbs.permute_reg_l1_norm > np.mean(cs_metrics_all_perturbs.permute_reg_l1_norm) + 2*np.std(cs_metrics_all_perturbs.permute_reg_l1_norm)]
            # Mahalanobis calculations

            #Correlations
            # pearson_correlations.append(pearsonr(cs_metrics.permute_reg_l1_scaled,cs_metrics.gt_mahalanobis)[0])
            # spearman_correlations.append(spearmanr(cs_metrics.permute_reg_l1_scaled,cs_metrics.gt_mahalanobis)[0])

            # How many of top 5, 10, and 32 drugs do we have?
            # top_drugs_by_gt_mahala = gt_metrics.gt_mahalanobis.sort_values(ascending=False).index.values
            # n = 5
            # top_5_mahala_percent.append(sum(np.isin(top_drugs_by_gt_mahala[0:n],cs_metrics.Metadata_perturbation))/n)
            # n = 10
            # top_10_mahala_percent.append(sum(np.isin(top_drugs_by_gt_mahala[0:n],cs_metrics.Metadata_perturbation))/n)
            # n = 32
            # top_32_mahala_percent.append(sum(np.isin(top_drugs_by_gt_mahala[0:n],cs_metrics.Metadata_perturbation))/n)


            # # GT leiden simple stats
            #     # num leiden in cs, fraction of leiden in cs, pvalue
            # num_leiden_in_cs = sum(cs_metrics.gt_sig_leiden==True)
            # number_of_gt_leiden_in_cs.append(num_leiden_in_cs)
            # num_total = 316
            # num_leiden_in_gt = sum(gt_metrics.gt_sig_leiden==True)
            # fraction_of_gt_leiden_in_cs.append(num_leiden_in_cs/num_leiden_in_gt)
            # num_cs_total = cs_metrics.shape[0]
            # pvalue_gt_leiden_in_cs.append(1-hypergeom.cdf(num_leiden_in_cs,num_total,num_leiden_in_gt,num_cs_total))
            # percent_of_cs_that_is_gt_leiden_sig.append(num_leiden_in_cs/cs_metrics.shape[0])
            
            
            # Sample saving stats
            # num_samples_cs = 316* 2 / cs_metrics.Metadata_compresion.iloc[0]
            # num_gt_reps = 6
            # num_samples_validate = cs_metrics.shape[0] * num_gt_reps
            # fraction_samples_total = (num_samples_cs + num_samples_validate)/(316*num_gt_reps)

            # cs_hit_nominate_num.append(cs_metrics.shape[0])
            # sample_savings.append(fraction_samples_total)

            run_vals.append(run)
            compression_vals.append(compression)
            rep_vals.append(replicates)
            scheme_vals.append(scheme)

            # n_drugs_per_pool.append(cs_metrics.Metadata_compresion.iloc[0]*cs_metrics.Metadata_replicates.iloc[0]/2)
            
            
            # just looking at hits two std dev above the mean
            # num_top_hits.append(top_cs_all.shape[0])
            # percent_top_sig_mahala.append(np.sum(top_cs_all.gt_signif_mahala)/top_cs_all.shape[0])
            # percent_top_sig_leiden.append(np.sum(top_cs_all.gt_sig_leiden)/top_cs_all.shape[0])
            # top_hit_thresholds.append(top_hit_threshold)

            # if cs_metrics.shape[0] > 0:

            total_positives = cs_metrics.shape[0]
            total_negatives = gt_metrics.shape[0] -total_positives

            true_positives = sum(cs_metrics.gt_signif_mahala==True)
            tp.append(true_positives)
            false_positives = total_positives - true_positives
            fp.append(false_positives)

            negative_data = gt_metrics.loc[np.isin(gt_metrics.Metadata_perturbation,cs_metrics.Metadata_perturbation,invert=True)]
            false_negatives = sum(negative_data.gt_signif_mahala=='True')
            fn.append(false_negatives)
            true_negatives = sum(negative_data.gt_signif_mahala!='True')
            tn.append(true_negatives)

            tpr.append(true_positives/(true_positives + false_negatives))

            fpr.append(false_positives/(false_positives + true_negatives))

            if permute_threshold == "0.0":
                 precision.append(-1)
                 recall.append(-1)
            else:
                precision.append(true_positives/(true_positives+false_positives))
                recall.append(true_positives/(true_positives+false_negatives))




    regression_hit_nomination_stats = pd.DataFrame({"Metadata_run":run_vals,
                                                   "Metadata_compression":compression_vals,
                                                   "Metadata_replicates":rep_vals,
                                                   "Metadata_scheme":scheme_vals,
                                                #    "pearson_gt_mahala_cs_perm_reg_l1_norm":pearson_correlations,
                                                #     "spearman_gt_mahala_cs_perm_reg_l1_norm":spearman_correlations,
                                                #    "percent_top5_mahala":top_5_mahala_percent,
                                                #    "percent_top10_mahala":top_10_mahala_percent,
                                                #    "percent_top32_mahala":top_32_mahala_percent,
                                                #    "num_gt_pheno_sig_in_cs":number_of_gt_leiden_in_cs,
                                                #    "frac_gt_pheno_sig_in_cs":fraction_of_gt_leiden_in_cs,
                                                #    "pval_gt_pheno_sig_in_cs":pvalue_gt_leiden_in_cs,
                                                #     "percent_of_cs_that_is_gt_leiden":percent_of_cs_that_is_gt_leiden_sig,
                                                #    "num_cs_hits":cs_hit_nominate_num,
                                                #    "sample_savings":sample_savings,
                                                #    'num_drugs_per_pool':n_drugs_per_pool,
                                                #    "num_top_hits":num_top_hits,
                                                #    "percent_top_sig_mahala":percent_top_sig_mahala,
                                                #    "percent_top_sig_leiden":percent_top_sig_leiden,
                                                #    "top_hit_threshold":top_hit_thresholds,
                                                   'tpr':tpr,
                                                   'fpr':fpr,
                                                   'tp':tp,
                                                   'fp': fp,
                                                   'tn':tn,
                                                   'fn':fn,
                                                  'precision':precision,
                                                   'recall':recall})     
    regression_hit_nomination_stats.to_csv("regression_summary_stats/summary_stats_from_all_reg_sig_coef_permute_"+permute_threshold+"_Dec102021.csv")

CS_run1
2.0_5.0random
2.0_7.0random
4.0_3.0random
4.0_5.0random
4.0_7.0random
8.0_3.0random
8.0_5.0random
8.0_7.0random
CS_run2
2.0_5.0random
2.0_7.0random
4.0_3.0random
4.0_5.0random
4.0_7.0random
8.0_3.0random
8.0_5.0random
8.0_7.0random
CS_run3
2.0_3.0random1
2.0_3.0random2
2.0_7.0random1
2.0_7.0random2
16.0_5.0random1
16.0_5.0random2
32.0_5.0random1
32.0_5.0random2
CS_run1
2.0_5.0random
2.0_7.0random
4.0_3.0random
4.0_5.0random
4.0_7.0random
8.0_3.0random
8.0_5.0random
8.0_7.0random
CS_run2
2.0_5.0random
2.0_7.0random
4.0_3.0random
4.0_5.0random
4.0_7.0random
8.0_3.0random
8.0_5.0random
8.0_7.0random
CS_run3
2.0_3.0random1
2.0_3.0random2
2.0_7.0random1
2.0_7.0random2
16.0_5.0random1
16.0_5.0random2
32.0_5.0random1
32.0_5.0random2
CS_run1
2.0_5.0random
2.0_7.0random
4.0_3.0random
4.0_5.0random
4.0_7.0random
8.0_3.0random
8.0_5.0random
8.0_7.0random
CS_run2
2.0_5.0random
2.0_7.0random
4.0_3.0random
4.0_5.0random
4.0_7.0random
8.0_3.0random
8.0_5.0random
8.0_7.0random
CS_run3
2.0_3.0r

In [32]:
fpr

[0.5581395348837209,
 0.43521594684385384,
 0.33554817275747506,
 0.6378737541528239,
 0.3089700996677741,
 0.4219269102990033,
 0.2558139534883721,
 0.14950166112956811,
 0.574750830564784,
 0.5382059800664452,
 0.6478405315614618,
 0.6179401993355482,
 0.5182724252491694,
 0.5448504983388704,
 0.3554817275747508,
 0.3588039867109635,
 0.46179401993355484,
 0.5049833887043189,
 0.3654485049833887,
 0.45182724252491696,
 0.08637873754152824,
 0.06976744186046512,
 0.10631229235880399,
 0.32558139534883723]

In [26]:
false_positives

0

In [25]:
true_positives+false_positives

0

In [21]:
true_positives/(true_positives+false_positives)

ZeroDivisionError: division by zero

In [22]:
true_positives+false_positives

0

In [24]:
0/15

0.0

In [20]:
true_negatives

301

In [11]:
print('done')

done


In [8]:
cs_metrics

,regression_l1_norm,permute_reg_l1_norm,permute_reg_l1_scaled,Metadata_perturbation,gt_mahalanobis,gt_signif_mahala,gt_leiden,gt_sig_leiden,Metadata_run,Metadata_compresion,Metadata_replicates,Metadata_scheme


In [9]:
permute_threshold

'0.0'

In [12]:
cs_metrics

,regression_l1_norm,permute_reg_l1_norm,permute_reg_l1_scaled,Metadata_perturbation,gt_mahalanobis,gt_signif_mahala,gt_leiden,gt_sig_leiden,Metadata_run,Metadata_compresion,Metadata_replicates,Metadata_scheme
0,40.270737,40.270737,0.329544,(S)-Timolol Maleate,3.124464,False,NaN,NaN,CS_run3,32.0,5.0,random2
1,54.465890,54.465890,0.445706,(±) Isoproterenol,4.260402,False,NaN,NaN,CS_run3,32.0,5.0,random2
4,28.774245,28.774245,0.235466,Acetylcholine Chloride,5.465217,False,6-8,True,CS_run3,32.0,5.0,random2
6,36.991442,36.991442,0.302709,Acyclovir (Acycloguanosine) Zovirax,4.081513,False,NaN,NaN,CS_run3,32.0,5.0,random2
9,25.936604,25.936604,0.212245,Alendronate,5.375833,False,NaN,NaN,CS_run3,32.0,5.0,random2
...,...,...,...,...,...,...,...,...,...,...,...,...
308,30.419897,30.419897,0.248932,Vorinostat,6.186531,False,NaN,NaN,CS_run3,32.0,5.0,random2
310,14.997181,14.997181,0.122725,"Zalcitabine (2',3'-Dideoxycytidine)",4.714244,False,NaN,NaN,CS_run3,32.0,5.0,random2
311,2.929907,2.929907,0.023976,Zidovudine (3'-Azido-3'-Deoxythymidine),5.843156,False,NaN,NaN,CS_run3,32.0,5.0,random2
312,28.582321,28.582321,0.233895,Zileuton,2.934890,False,NaN,NaN,CS_run3,32.0,5.0,random2


In [13]:
permute_threshold

'1.0'